In [16]:
import os
import pandas as pd
import re

# === Step 1: Define folder ===
folder = r"C:\Users\Rono\Downloads\AAAA Hackathon\Datasets\KNBS\Tables\National-Agriculture-Production-Report-2024_with_grids_and_borders"

# === Step 2: Collect matching Excel files ===
files = []
for fname in os.listdir(folder):
    if fname.lower().endswith(".xlsx") and ("_population of" in fname.lower() or "_population_of_" in fname.lower()):
        match = re.search(r"(20\d{2})", fname)
        if match:
            year = int(match.group(1))
            full_path = os.path.join(folder, fname)
            files.append((full_path, year))

print(f"✅ Found {len(files)} matching files with year.")

# === Step 3: Process each file ===
all_data = []

for file_path, year in files:
    try:
        print(f"\n🔍 Reading: {os.path.basename(file_path)}")
        df = pd.read_excel(file_path, header=0)
        df.columns = [str(col).strip() for col in df.columns]

        # Detect county column
        county_col = next((col for col in df.columns if "county" in col.lower()), None)
        if not county_col:
            print(f"⚠️ Skipping: No county column in {os.path.basename(file_path)}")
            continue

        # Melt into long format
        df_melted = df.melt(id_vars=[county_col], var_name="Livestock", value_name="Value")
        df_melted.rename(columns={county_col: "County"}, inplace=True)
        df_melted["year"] = year
        df_melted["SourceFile"] = os.path.basename(file_path)

        all_data.append(df_melted)
        print(f"📄 Processed: {os.path.basename(file_path)} ({len(df_melted)} rows)")

    except Exception as e:
        print(f"❌ Error processing {file_path}: {e}")

# === Step 4: Combine and save ===
if all_data:
    final_df = pd.concat(all_data, axis=0, ignore_index=True)
    final_df["County"] = final_df["County"].astype(str).str.strip()
    final_df["Livestock"] = final_df["Livestock"].astype(str).str.strip()

    print(f"\n✅ Final shape: {final_df.shape}")

    # Save
    output_path = os.path.join(folder, "all_livestock_population_vertical.xlsx")
    final_df.to_excel(output_path, index=False)
    print(f"💾 Saved vertical dataset to: {output_path}")
else:
    print("❌ No data extracted from the files.")


✅ Found 11 matching files with year.

🔍 Reading: Annex_15_Population of Cattle, Sheep and goats by county for the year 2021.xlsx
📄 Processed: Annex_15_Population of Cattle, Sheep and goats by county for the year 2021.xlsx (282 rows)

🔍 Reading: Annex_17_Population of Cattle, Sheep and goats by county for the year 2023.xlsx
📄 Processed: Annex_17_Population of Cattle, Sheep and goats by county for the year 2023.xlsx (288 rows)

🔍 Reading: Annex_18_Population_of_Donkeys_Camels_and_Beehives_Log_Kenya_2021.xlsx
📄 Processed: Annex_18_Population_of_Donkeys_Camels_and_Beehives_Log_Kenya_2021.xlsx (288 rows)

🔍 Reading: Annex_19_Population_of_Donkeys_Camels_and_Hives_Log_KTBH_Lan_2022.xlsx
📄 Processed: Annex_19_Population_of_Donkeys_Camels_and_Hives_Log_KTBH_Lan_2022.xlsx (336 rows)

🔍 Reading: Annex_20_Population_of_Donkeys_Camels_and_Hives_Log_KTBH_Lan_2023.xlsx
📄 Processed: Annex_20_Population_of_Donkeys_Camels_and_Hives_Log_KTBH_Lan_2023.xlsx (288 rows)

🔍 Reading: Annex_21_Population_of_Pi

In [24]:
import os
import pandas as pd
import re

# === Step 1: Define folder ===
folder = r"C:\Users\Rono\Downloads\AAAA Hackathon\Datasets\KNBS\Tables\National-Agriculture-Production-Report-2024_with_grids_and_borders"

# === Step 2: Collect matching Excel files ===
files = []
for fname in os.listdir(folder):
    if fname.lower().endswith(".xlsx") and "_livestock_products_by_county" in fname.lower():
        match = re.search(r"_for_year_(\d{4})", fname)
        if match:
            year = int(match.group(1))
            full_path = os.path.join(folder, fname)
            files.append((full_path, year))

print(f"✅ Found {len(files)} matching livestock product files with year.")

# === Step 3: Process each file ===
all_data = []

for file_path, year in files:
    try:
        print(f"\n🔍 Reading: {os.path.basename(file_path)}")
        df = pd.read_excel(file_path, header=0)
        df.columns = [str(col).strip() for col in df.columns]

        # Identify the county column
        county_col = next((col for col in df.columns if "county" in col.lower()), None)
        if not county_col:
            print(f"⚠️ Skipping: No county column in {os.path.basename(file_path)}")
            continue

        product_triplets = []
        cols = df.columns.tolist()
        i = cols.index(county_col) + 1  # start after county column

        while i + 2 < len(cols):
            qty_col = cols[i]
            unit_col = cols[i+1]
            total_col = cols[i+2]

            # Try to extract the product name from the first column
            match = re.match(r"(.+?)\s*\(kg\)", qty_col, re.IGNORECASE)
            if match:
                product_name = match.group(1).strip()
            else:
                product_name = qty_col.strip()

            product_triplets.append((product_name, qty_col, unit_col, total_col))
            i += 3

        # Now for each product, extract relevant values
        for product, qty_col, unit_col, total_col in product_triplets:
            subset = df[[county_col, qty_col, unit_col, total_col]].copy()
            subset.columns = ["County", "Quantity", "UnitPrice", "TotalValue"]
            subset["Product"] = product
            subset["Year"] = year
            subset["SourceFile"] = os.path.basename(file_path)

            all_data.append(subset)

        print(f"📄 Processed: {os.path.basename(file_path)} ({len(product_triplets)} products × {len(df)} rows)")

    except Exception as e:
        print(f"❌ Error processing {file_path}: {e}")

# === Step 4: Combine and Save ===
if all_data:
    final_df = pd.concat(all_data, axis=0, ignore_index=True)

    # Optional cleanup
    final_df["County"] = final_df["County"].astype(str).str.strip()
    final_df["Product"] = final_df["Product"].astype(str).str.strip()
    final_df = final_df[["County", "Product", "Quantity", "UnitPrice", "TotalValue", "Year", "SourceFile"]]

    print(f"\n✅ Final shape: {final_df.shape}")

    # Save
    output_path = os.path.join(folder, "all_livestock_products_long_format.xlsx")
    final_df.to_excel(output_path, index=False)
    print(f"💾 Saved long-format dataset to: {output_path}")
else:
    print("❌ No data extracted from the files.")


✅ Found 16 matching livestock product files with year.

🔍 Reading: Annex_27_Livestock_products_by_county_for_year_2021_p141.xlsx
📄 Processed: Annex_27_Livestock_products_by_county_for_year_2021_p141.xlsx (1 products × 48 rows)

🔍 Reading: Annex_27_Livestock_products_by_county_for_year_2021_p142.xlsx
📄 Processed: Annex_27_Livestock_products_by_county_for_year_2021_p142.xlsx (1 products × 48 rows)

🔍 Reading: Annex_27_Livestock_products_by_county_for_year_2021_p143.xlsx
📄 Processed: Annex_27_Livestock_products_by_county_for_year_2021_p143.xlsx (2 products × 48 rows)

🔍 Reading: Annex_27_Livestock_products_by_county_for_year_2021_p144.xlsx
📄 Processed: Annex_27_Livestock_products_by_county_for_year_2021_p144.xlsx (2 products × 48 rows)

🔍 Reading: Annex_27_Livestock_products_by_county_for_year_2021_p145.xlsx
📄 Processed: Annex_27_Livestock_products_by_county_for_year_2021_p145.xlsx (2 products × 48 rows)

🔍 Reading: Annex_27_Livestock_products_by_county_for_year_2021_p146.xlsx
📄 Processed